In [5]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
import MLModels


loc="../Clinical Data/Raw/"
outdata="../Clinical Data/Processed/"

In [3]:
import lifelines

In [4]:
def findcols(df,txt):
    cols=[col for col in df.columns if txt in col]
    return cols

def datacleaner(df):
    for col in df.columns:
        masknotna=(df[col][pd.notnull(df[col])])
        df[col]=df[col].fillna(df[col][masknotna].mean())
    return df
    

In [6]:
udprs_cog_moc=pd.read_csv('%s%s' % (outdata,'udprs_cog_moc.csv'))

In [52]:
udprs_cog_moc.groupby(['SYDID','Date of assessment'])['FOGQ 3'].mean()

SYDID        Date of assessment
PD_00001_DA  2013-05-28            NaN
PD_00002_MW  2011-10-25            3.0
PD_00004_AW  2011-02-01            0.0
             2012-02-21            0.0
             2013-02-12            0.0
                                  ... 
PD_00553_KM  2012-10-16            0.0
PD_00555_PH  2012-11-20            0.0
PD_00557_DC  2012-11-06            NaN
PD_00568_MB  2012-11-27            0.0
PD_00583_KC  2013-03-19            NaN
Name: FOGQ 3, Length: 263, dtype: float64

In [49]:
maskfog=(udprs_cog_moc['FOGQ 3']>0)
masknofog=(udprs_cog_moc['FOGQ 3']==0)

udprs_cog_moc['fog']=0
udprs_cog_moc['fog'][maskfog]=1
udprs_cog_moc['fog']=udprs_cog_moc.groupby('SYDID')['fog'].transform('max')

udprs_cog_moc['FOG_Date']=pd.to_datetime\
(udprs_cog_moc[maskfog].groupby('SYDID')['Date of assessment'].transform('min'))




A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [50]:
udprs_cog_moc['fog'].value_counts()

0    159
1    107
Name: fog, dtype: int64

In [33]:
maskfog=(udprs_cog_moc['FOGQ 3']>0)
masknofog=(udprs_cog_moc['FOGQ 3']==0)

udprs_cog_moc['fog']=0
udprs_cog_moc['fog'][maskfog]=1
udprs_cog_moc['fog']=udprs_cog_moc.groupby('SYDID')['fog'].transform('max')

masknofog2=(udprs_cog_moc['no_fog']==1)





A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [36]:
findcols(udprs_cog_moc,'Diag')

['Princ_Diag', 'Date Diagnosed', 'Age_Diagnosis']

In [45]:
udprs_cog_moc['time_to_fog']=(pd.to_datetime(udprs_cog_moc['FOG_Date'])-pd.to_datetime(udprs_cog_moc['Date Diagnosed'])).dt.days


#[pd.notnull(udprs_cog_moc['FOG_Date'])].mean()

In [47]:
udprs_cog_moc['time_to_fog'][pd.notnull(udprs_cog_moc['time_to_fog'])]

1      1546.0
9      8101.0
10     7139.0
11     7139.0
16     4263.0
        ...  
241    2562.0
242    1554.0
244    7264.0
251    3176.0
256    4651.0
Name: time_to_fog, Length: 75, dtype: float64

In [22]:
udprs_cog_moc['MCI']=0
mask=(udprs_cog_moc['MoCA Total']<27)
udprs_cog_moc['MCI'][mask]=1
mask2=(udprs_cog_moc['MCI']==1)
udprs_cog_moc['MCI_date']=udprs_cog_moc[mask2].groupby('SYDID')['Date of assessment'].transform('min')




A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [31]:
pd.to_datetime(udprs_cog_moc['MCI_date']).mean()

Timestamp('2012-01-17 00:45:28.421052672')

In [30]:
pd.to_datetime(udprs_cog_moc['Date of assessment']).mean()

Timestamp('2012-09-09 18:13:32.030075136')